In [36]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [37]:
df = pd.read_csv("../../pre_processing/datasets/no_filtered_analysis.csv", sep = ",")

In [38]:
if "RA" in df.columns:
    print("Removing column RA")
    df.drop('RA', inplace=True, axis=1)

In [39]:
df.head()

,EVADIDO,GRADE_CORRENTE,NOTA_MEDIA,TOT_MAT_CURSADAS,TOT_APROVACOES,TOT_REPROVACOES,TOT_REPROV_FREQ,TOT_REPROV_NOTA_FREQ,NCS005,NCS012,...,SIF068,SIF029,SIF030,SIF038,SIF070,NCS037,NCS040,SIF006,SIF072,SIF073
0,0,5,7.0,10,9,1,0,1,0,0,...,0,4,0,0,0,0,0,0,1,1
1,1,4,5.5,18,15,3,3,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,1,3,3.0,12,6,6,0,6,0,1,...,0,0,0,0,0,0,0,4,0,0
3,1,3,5.1,49,37,12,10,2,0,1,...,0,1,1,5,1,1,1,1,0,0
4,0,5,5.4,10,8,2,2,0,0,0,...,0,0,0,0,0,0,0,0,2,1


In [40]:
X = df.iloc[:, df.columns != "EVADIDO"].values
y = df["EVADIDO"]

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [42]:
clf = DecisionTreeClassifier(criterion="gini", splitter="best", max_depth=10)

In [43]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [44]:
predictions = clf.predict(X_test)

In [45]:
predictions

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [46]:
np.asarray(y_test)

array([0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0], dtype=int64)

In [47]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [48]:
print(confusion_matrix(y_test, predictions))

[[91 10]
 [ 6 60]]


In [49]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92       101
           1       0.86      0.91      0.88        66

   micro avg       0.90      0.90      0.90       167
   macro avg       0.90      0.91      0.90       167
weighted avg       0.91      0.90      0.90       167



In [50]:
print("Model accuracy: ", accuracy_score(y_test, predictions), "%")

Model accuracy:  0.9041916167664671 %


In [51]:
from sklearn import tree
import graphviz 

In [52]:
feature_names = df.drop("EVADIDO", axis = 1)
feature_names = feature_names.columns
class_names = ["EVADIDO","NÃO EVADIDO"]

dot_data = tree.export_graphviz(clf, out_file = None,
                    feature_names=feature_names,
                    class_names=class_names,
                    filled=True, 
                    rounded=True,
                    special_characters=True,
                    precision=3)

graph = graphviz.Source(dot_data)
graph.format = "png"
graph.render('dtree_render',view=True)

'dtree_render.png'